In [6]:
import sqlite3
# from pxr import Usd, UsdGeom, Gf, Vt, Sdf
def read_data_from_sqlite(db_file, query):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_file)

    # Create a cursor object to execute queries
    cursor = conn.cursor()

    # Execute the query
    cursor.execute(query)

    # Fetch all the results from the query
    results = cursor.fetchall()

    # Close the cursor and the connection
    cursor.close()
    conn.close()

    return results


database_file = "geometry_atlas.db"

AlignableTransforms_query = "SELECT * FROM AlignableTransforms"
ChildrenPositions_query = "SELECT * FROM ChildrenPositions"
Elements_query = "SELECT * FROM Elements"
FullPhysVols_query = "SELECT * FROM FullPhysVols"
Functions_query = "SELECT * FROM Functions"
GeoNodesTypes_query = "SELECT * FROM GeoNodesTypes"
IdentifierTags_query = "SELECT * FROM IdentifierTags"
LogVols_query = "SELECT * FROM LogVols"
Materials_query = "SELECT * FROM Materials"
NameTags_query = "SELECT * FROM NameTags"
PhysVols_query = "SELECT * FROM PhysVols"
RootVolume_query = "SELECT * FROM RootVolume"
SerialDenominators_query = "SELECT * FROM SerialDenominators"
SerialIdentifiers_query = "SELECT * FROM SerialIdentifiers"
SerialTransformers_query = "SELECT * FROM SerialTransformers"
Shapes_query = "SELECT * FROM Shapes"
Transforms_query = "SELECT * FROM Transforms"

# Call the function to read data from the SQLite file
AlignableTransforms_data = read_data_from_sqlite(database_file, Shapes_query)
ChildrenPositions_data = read_data_from_sqlite(database_file, ChildrenPositions_query)
print(f'ChildrenPositions:{len(ChildrenPositions_data)}')
Elements_data = read_data_from_sqlite(database_file, Elements_query)
FullPhysVols_data = read_data_from_sqlite(database_file, FullPhysVols_query)
print(f'FullPhysVols:{len(FullPhysVols_data)}')
Functions_data = read_data_from_sqlite(database_file, Functions_query)
GeoNodesTypes_data = read_data_from_sqlite(database_file, GeoNodesTypes_query)
IdentifierTags_data = read_data_from_sqlite(database_file, IdentifierTags_query)
LogVols_data = read_data_from_sqlite(database_file, LogVols_query)
print(f'LogVols:{len(LogVols_data)}')
Materials_data = read_data_from_sqlite(database_file, Materials_query)
NameTags_data = read_data_from_sqlite(database_file, NameTags_query)
PhysVols_data = read_data_from_sqlite(database_file, PhysVols_query)
print(f'PhysVols:{len(PhysVols_data)}')
RootVolume_data = read_data_from_sqlite(database_file, RootVolume_query)
print(f'RootVols:{len(RootVolume_data)}')
SerialDenominators_data = read_data_from_sqlite(database_file, SerialDenominators_query)
SerialIdentifiers_data = read_data_from_sqlite(database_file, SerialIdentifiers_query)
SerialTransformers_data = read_data_from_sqlite(database_file, SerialTransformers_query)
print(len(SerialTransformers_data))
Shapes_data = read_data_from_sqlite(database_file, Shapes_query)
print(len(Shapes_data))
Transforms_data = read_data_from_sqlite(database_file, Transforms_query)
print(len(Transforms_data))

print('Root volume', RootVolume_data)
# Print the retrieved data
print('Logical Volumes')
for r in range(3):
   print(LogVols_data[r])

print('Full Physical Volumes')
for r in range(3):
   print(FullPhysVols_data[r])

print('Physical Volumes')
for r in range(3):
   print(PhysVols_data[r])


ChildrenPositions:617000
FullPhysVols:26144
LogVols:58719
PhysVols:57675
RootVols:1
5214
119487
186717
Root volume [(1, 1, 1)]
Logical Volumes
(1, 'WorldLog', 1, 1)
(2, 'Pixel', 2, 1)
(3, 'Barrel', 3, 1)
Full Physical Volumes
(1, 2)
(2, 3)
(3, 4)
Physical Volumes
(1, 1)
(2, 5)
(3, 8)



* LogVols - name, shapE, materiaL (59k) (3127 distinct names)
* PhysVols - id, logvoL (58k)
* FullPhysVol - id, logvoL (26k)
* Shapes - id, TYPE, parameters (119k)
* Materials - id, name, density, elements (522)
* Elements - id, name, symbol, Z, A (42)

* RootVolume - id, volId, volTable (1,1,1)
* GeoNodesTypes - id, nodeType, tableName (14)
* NameTags - id, name (47k)
* ChildrenPositions - id, parentId, parentTable, parentCopyNumber, position, childTable, childId, childCopyNumber (617k)

In [35]:
NT={}
for nt in NameTags_data: NT[nt[0]]={'name':nt[1]}
print(NT[1])
FPV={}
for fpv in FullPhysVols_data: FPV[fpv[0]]={'logvol':fpv[1]}
print(FPV[1])
LV={}
for lv in LogVols_data: LV[lv[0]]={'name':lv[1], 'shape':lv[2], 'material': lv[3]}
print(LV[1])
GNT={}
for gnt in GeoNodesTypes_data: GNT[gnt[0]]={'nodeType':gnt[1], 'tableName': gnt[2]}
print(GNT)
CP={}
for cp in ChildrenPositions_data: CP[cp[0]]={'parentId':cp[1], 'parentTable': cp[2],'parentTable': cp[2],'parentCopyNumber': cp[3],
                                             'position': cp[4],'childTable': cp[5],'childId': cp[6], 'childCopyNumber': cp[7]}
print(CP[1])

{'name': 'Pixel'}
{'logvol': 2}
{'name': 'WorldLog', 'shape': 1, 'material': 1}
{1: {'nodeType': 'GeoPhysVol', 'tableName': 'PhysVols'}, 2: {'nodeType': 'GeoFullPhysVol', 'tableName': 'FullPhysVols'}, 3: {'nodeType': 'GeoLogVol', 'tableName': 'LogVols'}, 4: {'nodeType': 'GeoMaterial', 'tableName': 'Materials'}, 5: {'nodeType': 'GeoElement', 'tableName': 'Elements'}, 6: {'nodeType': 'GeoShape', 'tableName': 'Shapes'}, 7: {'nodeType': 'GeoSerialDenominator', 'tableName': 'SerialDenominators'}, 8: {'nodeType': 'GeoSerialIdentifier', 'tableName': 'SerialIdentifiers'}, 9: {'nodeType': 'GeoIdentifierTag', 'tableName': 'IdentifierTags'}, 10: {'nodeType': 'Function', 'tableName': 'Functions'}, 11: {'nodeType': 'GeoSerialTransformer', 'tableName': 'SerialTransformers'}, 12: {'nodeType': 'GeoTransform', 'tableName': 'Transforms'}, 13: {'nodeType': 'GeoAlignableTransform', 'tableName': 'AlignableTransforms'}, 14: {'nodeType': 'GeoNameTag', 'tableName': 'NameTags'}}
{'parentId': 1, 'parentTable': 

children pos:
1    1   physvol 1 1 nametag 

In [38]:
from copy import deepcopy


tree={}
nodes=[]
for cp in CP.values():
    if cp['parentId']==1 and cp['parentTable']==1: # parentId=1
        tcp=deepcopy(cp)
        tcp['parentTable']=GNT[cp['parentTable']]['tableName']
        tcp['childTable']=GNT[cp['childTable']]['tableName']
        cid=cp['childId']
        if tcp['childTable']=='NameTags': tcp['childName']=NT[cid]['name']
        if tcp['childTable']=='FullPhysVols': tcp['childLogVol']=LV[FPV[cid]['logvol']]
        print(tcp)
        nodes.append(tcp)
print(len(nodes))

{'parentId': 1, 'parentTable': 'PhysVols', 'parentCopyNumber': 1, 'position': 1, 'childTable': 'NameTags', 'childId': 1, 'childCopyNumber': 0, 'childName': 'Pixel'}
{'parentId': 1, 'parentTable': 'PhysVols', 'parentCopyNumber': 1, 'position': 2, 'childTable': 'AlignableTransforms', 'childId': 1, 'childCopyNumber': 0}
{'parentId': 1, 'parentTable': 'PhysVols', 'parentCopyNumber': 1, 'position': 3, 'childTable': 'FullPhysVols', 'childId': 1, 'childCopyNumber': 1, 'childLogVol': {'name': 'Pixel', 'shape': 2, 'material': 1}}
{'parentId': 1, 'parentTable': 'PhysVols', 'parentCopyNumber': 1, 'position': 4, 'childTable': 'NameTags', 'childId': 4905, 'childCopyNumber': 0, 'childName': 'SCT'}
{'parentId': 1, 'parentTable': 'PhysVols', 'parentCopyNumber': 1, 'position': 5, 'childTable': 'IdentifierTags', 'childId': 4294, 'childCopyNumber': 0}
{'parentId': 1, 'parentTable': 'PhysVols', 'parentCopyNumber': 1, 'position': 6, 'childTable': 'AlignableTransforms', 'childId': 2036, 'childCopyNumber': 0

Need to work on building the tree：
1. Build from ChildrenPosition relationship


Parents are FullPhysVols or PhysVols, should I build a dict for the relationship? What is childCopyNumber?
Combination is done using SerialDenominators?


2. FullPhysVols, PhysVols ->LogVols ->Shapes/Materials


Test Code: Creating Box

In [28]:
#Test creating box
    path = "assests/shapes/shape_"+ str(Parameters[0])+".usda"
    shape_name = "/shape_"+str(Parameters[0])
    stage = Usd.Stage.CreateNew(path)
    xform = UsdGeom.Xform.Define(stage, shape_name)
    box = UsdGeom.Cube.Define(stage, shape_name+"/geometry")

    key_value_pairs=Parameters[2].split(";")
    box_length=[]
    for value in key_value_pairs:
        box_length.append(float(value.split("=")[1])*2)
    extent = Vt.Vec3fArray(1, Gf.Vec3f(box_length[0],box_length[1],box_length[2]))
    box.GetPrim().CreateAttribute("extent", Sdf.ValueTypeNames.Float3Array).Set(extent)

    # Add the box as a child of the xform
    xform.GetPrim().GetChildren().append(box.GetPrim())

    # Save the USD file
    stage.GetRootLayer().Save()
    
print(Shapes_data[10][2].split(";")[0])
print(Shapes_data[7])
exec(Shapes_data[10][2].split(";")[0])
exec(Shapes_data[10][2].split(";")[1])
print(Shapes_data[opA-1],Shapes_data[opB-1])


(33, 'Box', 'XHalfLength=0.0650000000000000;YHalfLength=4.3600000000000003;ZHalfLength=9.2349999999999994')
[0.13, 8.72, 18.47]


In [84]:
def Create_Single_Shape(Parameters):
    if Parameters[1] == "Tube":
        Create_Tube(Parameters)
    elif Parameters[1] == "Box":
        Create_Box(Parameters)
    elif Parameters[1] == "SimplePolygonBrep":
        Create_SimplePolygonBrep(Parameters)
    elif Parameters[1] == "Trap":
        Create_Trap(Parameters)
    elif Parameters[1] == "Tubs":
        Create_Tubs(Parameters)
    elif Parameters[1] == "Trd":
        Create_Trd(Parameters)
    elif Parameters[1] == "Pgon":
        Create_Pgon(Parameters)
    elif Parameters[1] == "Cons":
        Create_Cons(Parameters)
    elif Parameters[1] == "Para":
        Create_Para(Parameters)
    elif Parameters[1] == "Pcon":
        Create_Pcon(Parameters)
    elif Parameters[1] == "UnidentifiedShape":
        Create_UnidentifiedShape(Parameters)
    else:
        print("Error with creating Single Shape")

In [85]:
def Create_Tube(Parameters):
    return
def Create_Box(Parameters):
    path = "assests/shapes/shape_"+ str(Parameters[0])+".usda"
    shape_name = "/shape_"+str(Parameters[0])
    
    stage = Usd.Stage.CreateNew(path)
    xform = UsdGeom.Xform.Define(stage, shape_name)
    box = UsdGeom.Cube.Define(stage, shape_name+"/geometry")

    key_value_pairs=Parameters[2].split(";")
    box_length=[]
    for value in key_value_pairs:
        box_length.append(float(value.split("=")[1])*2)
    extent = Vt.Vec3fArray(1, Gf.Vec3f(box_length[0],box_length[1],box_length[2]))
    box.GetPrim().CreateAttribute("extent", Sdf.ValueTypeNames.Float3Array).Set(extent)

    # Add the box as a child of the xform
    xform.GetPrim().GetChildren().append(box.GetPrim())

    # Save the USD file
    stage.GetRootLayer().Save()
    return
def Create_SimplePolygonBrep(Parameters):
    return
def Create_Trap(Parameters):
    return
def Create_Tubs(Parameters):
    return
    
def Create_Trd(Parameters):
    return
    
def Create_Pgon(Parameters):
    return
    
def Create_Cons(Parameters):
    return
    
def Create_Para(Parameters):
    return
    
def Create_Pcon(Parameters):
    return
    
def Create_UnidentifiedShape(Parameters):
    return


In [89]:
def Combine_Two_Shapes(Parameters,Shapes_data):
    #print(Parameters[1]+ " Two Shapes " +Parameters[2].split(";")[0] + " "+Parameters[2].split(";")[1])
    if Parameters[1] == "Shift":
        Shift_Shape(Parameters,Shapes_data)
    elif Parameters[1] =="Union":
        Union_Shape(Parameters,Shapes_data)
    elif Parameters[1] =="Subtraction":
        Subtraction_Shape(Parameters,Shapes_data)
    elif Parameters[1] =="Intersection":
        Intersection_Shape(Parameters,Shapes_data)
    else:
        print("Error with Combine two shapes")

In [90]:
def Shift_Shape(Parameters,Shapes_data):
    return
def Union_Shape(Parameters,Shapes_data):
    return
def Subtraction_Shape(Parameters,Shapes_data):
    return
def Intersection_Shape(Parameters,Shapes_data):
    return

In [91]:
for row in data:
    if row[1] in {"Tube","Box","SimplePolygonBrep","Trap","Tubs","Trd","Pgon","Cons","Para","Pcon","UnidentifiedShape"}:
        Create_Single_Shape(row)
    elif row[1] in {"Shift","Union","Subtraction","Intersection"}:
        Combine_Two_Shapes(row,Shapes_data)        
    else:
        print("Error with creating shape")